# probability of the real structure,  under different SHAPE arrays

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)
from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

# Load data

In [ ]:
import ShaKer.rna_tools.rna_io as rio

def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}

data = getdata()

for k,v in data.items():print k,"\t",  len(v[1])

# TRAIN , get proba of structures

In [ ]:
from ShaKer.rna_tools.sukosd import sukosd
from ShaKer.rna_tools.structureprobability import probabilities_of_structures
import ShaKer.simushape as sim

data = getdata()
model  = sim.make_model(data,['23sRNA','R009'], model= sim.make_forestregressor())
data.pop("23sRNA")
data.pop("R009")

def get_str(seq):
    for _, data_seq, db in data.values():
        if seq == data_seq:
            return [db]
    print "SHIT"

#predictions = { k: (v[1],sim.predict(model,v[1],seq_to_db_function=get_str)) for k,v in data.items()  }
predictions = { k: (v[1],sim.predict(model,v[1])) for k,v in data.items()  }

keys = predictions.keys()


prb_suko =   [probabilities_of_structures(data[k][1], [data[k][2]], sukosd(data[k][2]))[0][1] for  k in keys] # input: seq and str, returns probability (sukosd(data[k][2]) returns shape)
prb_real =   [probabilities_of_structures(data[k][1],[data[k][2]], data[k][0])[0][1] for  k in keys] #rnaplfold gets sequence and shape, returns probab 
prb_shaker = [probabilities_of_structures(data[k][1],[data[k][2]], predictions[k])[0][1] for k in keys] # a predcition is seq,rea # probability




# Visualize 

In [ ]:
import pandas
import numpy as np
index= ['suko',"shaker", "real"]
data2 = [prb_suko,prb_shaker, prb_real ] # data is a list 

map(lambda x:x.append(np.mean(x)),data2)
df = pandas.DataFrame(data2, columns=keys+['mean'], index=index).T
df

# BUT

this is  not mega nice. 
so ... 
1. try to show energies instead of proba, because the probas seem messed up.. 
2. compare with no-data given

In [ ]:
from ShaKer.rna_tools.structureprobability import get_stru_energy

prb_suko =   [get_stru_energy(data[k][2],data[k][1], sukosd(data[k][2])) for  k in keys] # input: seq and str, returns probability (sukosd(data[k][2]) returns shape)
prb_real =   [get_stru_energy(data[k][2],data[k][1], data[k][0]) for  k in keys] #rnaplfold gets sequence and shape, returns probab 
prb_shaker = [get_stru_energy(data[k][2],data[k][1], predictions[k]) for k in keys] # a predcition is seq,rea # probability
prb_nodata = [get_stru_energy(data[k][2],data[k][1]) for k in keys] # a predcition is seq,rea # probability



In [ ]:
import pandas
import numpy as np
index= ['suko',"shaker", "real",'nothing']
#data = [corr_suko,corr_shaker]
data2 = [prb_suko,prb_shaker, prb_real,prb_nodata ] # data is a list 

map(lambda x:x.append(np.mean(x)),data2)
df = pandas.DataFrame(data2, columns=keys+['mean'], index=index).T

In [ ]:
print df